# Atlas Vector Search - Create Embeddings - Open Source - New Data

This notebook is a companion for the [Create Embeddings](https://www.mongodb.com/docs/atlas/atlas-vector-search/create-embeddings/) page. Refer to the page for set up steps and explanation details.

This notebook generates embeddings using an open source model for new data that is not already present in Atlas.

In [ ]:
pip install --quiet sentence-transformers pymongo einops

In [ ]:
from sentence_transformers import SentenceTransformer

# Load the embedding model (https://huggingface.co/nomic-ai/nomic-embed-text-v1")
model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

# Define a function to generate embeddings
def get_embedding(data):
   """Generates vector embeddings for the given data."""

   embedding = model.encode(data)
   return embedding.tolist()

# Generate an embedding
get_embedding("foo")

In [ ]:
import pymongo

# Connect to your Atlas cluster
mongo_client = pymongo.MongoClient("<connection-string>")
db = mongo_client["sample_db"]
collection = db["embeddings"]

# Sample data
data = [
   "Titanic: The story of the 1912 sinking of the largest luxury liner ever built",
   "The Lion King: Lion cub and future king Simba searches for his identity",
   "Avatar: A marine is dispatched to the moon Pandora on a unique mission"
]

# Ingest data into Atlas
inserted_doc_count = 0
for text in data:
   embedding = get_embedding(text)
   collection.insert_one({ "text": text, "embedding": embedding })
   inserted_doc_count += 1

print(f"Inserted {inserted_doc_count} documents.")

In [ ]:
from pymongo.operations import SearchIndexModel

# Create your index model, then create the search index
search_index_model = SearchIndexModel(
  definition = {
    "fields": [
      {
        "type": "vector",
        "path": "embedding",
        "similarity": "euclidean",
        "numDimensions": 768
      }
    ]
  },
  name="vector_index",
  type="vectorSearch",
)
collection.create_search_index(model=search_index_model)

In [ ]:
# Generate embedding for the search query
query_embedding = get_embedding("ocean tragedy")

# Sample vector search pipeline
pipeline = [
   {
      "$vectorSearch": {
            "index": "vector_index",
            "queryVector": query_embedding,
            "path": "embedding",
            "exact": True,
            "limit": 5
      }
   }, 
   {
      "$project": {
         "_id": 0, 
         "text": 1,
         "score": {
            "$meta": "vectorSearchScore"
         }
      }
   }
]

# Execute the search
results = collection.aggregate(pipeline)

# Print results
for i in results:
   print(i)
